In [1]:
import pandas as pd
import importlib as il
import sys, os

src = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src)

# project modules
import ingest.ingest as ing

In [18]:
il.reload(ing)
data_dir = '../ingest/data_in/'
schema_file = '../ingest/schema.json'
ingest = ing.Ingest(schema_file)
parsed_datasets = ingest.read_and_parse(data_dir)

Dataset: areas_countries_data
iso3_code     string
m2_area      float64
dtype: object
(256, 2)
***

Dataset: areas_zones_data
iso3_code        string
climate_zone      int64
m2_area         float64
dtype: object
(366, 3)
***

Dataset: building_statistics_fin
observation_year      int64
gross_floor_area    float64
dtype: object
(6, 2)
***

Dataset: climatetrace_countries
country_name    string
iso3_code       string
dtype: object
(252, 2)
***

Dataset: countries_data
iso3_code           string
pop_density        float64
emission_factor    float64
dtype: object
(256, 3)
***

Dataset: heating_demand_data
climate_zone               int64
heating_demand_factor    float64
dtype: object
(10, 2)
***

Dataset: manual_continents_data
iso3_code         string
continent_name    string
dtype: object
(27, 2)
***

Dataset: ne_countries_continents
continent_name    string
iso3_code         string
dtype: object
(255, 2)
***

Dataset: on_site_heat_data
continent_name     string
oheg_factor       float64

In [3]:
print(ingest.ds_names)
areas_countries_data = parsed_datasets['areas_countries_data']
areas_zones_data = parsed_datasets['areas_zones_data']
building_statistics_fin = parsed_datasets['building_statistics_fin']
climatetrace_countries = parsed_datasets['climatetrace_countries']
countries_data = parsed_datasets['countries_data']
heating_demand_data = parsed_datasets['heating_demand_data']
manual_continents_data = parsed_datasets['manual_continents_data']
ne_countries_continents = parsed_datasets['ne_countries_continents']
on_site_heat_data = parsed_datasets['on_site_heat_data']

['areas_countries_data', 'areas_zones_data', 'building_statistics_fin', 'climatetrace_countries', 'countries_data', 'heating_demand_data', 'manual_continents_data', 'ne_countries_continents', 'on_site_heat_data']


In [4]:
areas_countries_data

,iso3_code,m2_area
0,LBY,1.616064e+12
1,CRI,5.117676e+10
2,LBN,1.023806e+10
3,VIR,3.621598e+08
4,LBR,9.591809e+10
...,...,...
251,RWA,2.527768e+10
252,PAN,7.510388e+10
253,TZA,9.405763e+11
254,ZMB,7.504916e+11


In [5]:
areas_zones_data

,iso3_code,climate_zone,m2_area
0,LBY,3,8.928505e+11
1,LBY,1,2.291252e+10
2,LBY,2,7.003007e+11
3,CRI,1,5.113669e+10
4,LBN,3,1.023806e+10
...,...,...,...
361,PAN,1,7.505592e+10
362,TZA,1,9.323747e+11
363,ZMB,1,7.504916e+11
364,AGO,1,1.237890e+12


In [6]:
building_statistics_fin

,observation_year,gross_floor_area
0,2015,466737473.0
1,2016,474052712.0
2,2017,482457803.0
3,2018,488370838.0
4,2019,494625956.0
5,2020,505285969.0


In [7]:
climatetrace_countries

,country_name,iso3_code
0,Aruba,ABW
1,Afghanistan,AFG
2,Angola,AGO
3,Anguilla,AIA
4,Åland Islands,ALA
...,...,...
247,Samoa,WSM
248,Yemen,YEM
249,South Africa,ZAF
250,Zambia,ZMB


In [8]:
countries_data

,iso3_code,pop_density,emission_factor
0,ABW,0.000624,199.4
1,AFG,0.000049,199.4
2,AGO,0.000023,199.4
3,AIA,0.000140,199.4
4,ALA,0.000019,199.4
...,...,...,...
251,XSP,0.000000,199.4
252,YEM,0.000064,199.4
253,ZAF,0.000048,199.4
254,ZMB,0.000022,199.4


In [9]:
heating_demand_data

,climate_zone,heating_demand_factor
0,1,0.000020
1,2,0.000023
2,3,0.000052
3,4,0.000133
4,5,0.000160
5,6,0.000241
6,7,0.000159
7,8,0.000230
8,9,0.000385
9,10,0.000245


In [10]:
manual_continents_data

,iso3_code,continent_name
0,ALA,Northern Europe
1,BES,North America
2,BVT,Seven seas (open ocean)
3,CCK,Seven seas (open ocean)
4,CXR,Seven seas (open ocean)
5,ESH,Africa
6,GLP,North America
7,GUF,South America
8,MTQ,North America
9,MYT,Africa


In [11]:
ne_countries_continents

,continent_name,iso3_code
0,Asia,IDN
1,Asia,MYS
2,South America,CHL
3,South America,BOL
4,South America,PER
...,...,...
250,Asia,MAC
251,Oceania,ATC
252,North America,BJN
253,North America,SER


In [12]:
on_site_heat_data

,continent_name,oheg_factor,hfa_factor,ohfa_factor
0,Africa,0.9,0.5,0.45
1,Antarctica,0.0,0.0,0.00
2,Asia,0.8,0.7,0.56
3,Europe,0.6,0.9,0.54
4,<NA>,0.0,0.0,0.00
5,North America,0.7,0.9,0.63
6,Northern Europe,0.3,0.9,0.27
7,Oceania,0.8,0.5,0.40
8,Seven seas (open ocean),0.0,0.0,0.00
9,South America,0.8,0.7,0.56


In [13]:
countries_data = parsed_datasets['countries_data']
manual_continents_data = parsed_datasets['manual_continents_data']
countries_data.join(manual_continents_data.set_index('iso3_code')['continent_name'], on='iso3_code')

,iso3_code,pop_density,emission_factor,continent_name
0,ABW,0.000624,199.4,<NA>
1,AFG,0.000049,199.4,<NA>
2,AGO,0.000023,199.4,<NA>
3,AIA,0.000140,199.4,<NA>
4,ALA,0.000019,199.4,Northern Europe
...,...,...,...,...
251,XSP,0.000000,199.4,Asia
252,YEM,0.000064,199.4,<NA>
253,ZAF,0.000048,199.4,<NA>
254,ZMB,0.000022,199.4,<NA>
